In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [174]:
tes = pd.read_csv('tes.csv',sep=';',usecols = ['11','258','260','264','294'])

In [175]:
tes

,11,258,260,264,294
0,2,5,1,2,0
1,0,3,0,0,1
2,0,2,4,2,0
3,4,5,4,3,5
4,0,2,0,3,2
5,3,4,1,4,1
6,3,5,3,0,3
7,4,0,0,2,3


In [176]:
dataarray = np.array(tes)
print(dataarray)

[[2 5 1 2 0]
 [0 3 0 0 1]
 [0 2 4 2 0]
 [4 5 4 3 5]
 [0 2 0 3 2]
 [3 4 1 4 1]
 [3 5 3 0 3]
 [4 0 0 2 3]]


In [177]:
first_centroids = np.array([[4,5,4,3,5],[3,4,1,4,1]])
kmeans = KMeans(n_clusters=2,init=first_centroids)
cluster = kmeans.fit_predict(dataarray)
cluster

array([1, 1, 1, 0, 1, 1, 0, 1])

In [178]:
cluster0 = []
cluster1 = []
cluster2 = []
for i, j in enumerate(cluster):
    if j == 1:
        cluster1.append(matrix[i])
    else:
        cluster0.append(matrix[i])
print('cluster0=',cluster0)
print('cluster1=',cluster1)

cluster0= [[4, 5, 4, 3, 5], [3, 5, 3, 0, 3]]
cluster1= [[2, 5, 1, 2, 0], [0, 3, 0, 0, 1], [0, 2, 4, 2, 0], [0, 2, 0, 3, 2], [3, 4, 1, 4, 1], [4, 0, 0, 2, 3]]


In [179]:
clusterhitung = cluster1
average = []
for i in clusterhitung:
    count = 0
    avg = 0
    for j in i:
        if j != 0:
            avg += j
            count += 1
    average.append(avg/count)
print(average)

[2.5, 2.0, 2.6666666666666665, 2.3333333333333335, 2.6, 3.0]


In [180]:
mcr = []
for i in range(len(clusterhitung)):
    data = []
    for j in range(len(clusterhitung[i])):
        if clusterhitung[i][j] != 0:
            data.append(clusterhitung[i][j]-average[i])
        else:
            data.append(clusterhitung[i][j])
    mcr.append(data)
mcr

[[-0.5, 2.5, -1.5, -0.5, 0],
 [0, 1.0, 0, 0, -1.0],
 [0, -0.6666666666666665, 1.3333333333333335, -0.6666666666666665, 0],
 [0, -0.3333333333333335, 0, 0.6666666666666665, -0.3333333333333335],
 [0.3999999999999999, 1.4, -1.6, 1.4, -1.6],
 [1.0, 0, 0, -1.0, 0.0]]

In [181]:
target = matrix[1]
print(target)
sim = []
for i in range(6):
    data = []
    for j in range(1):
        print(i)
        print(clusterhitung[2])
        print(clusterhitung[i])
        # if clusterhitung[j][i] != 0:
        #     data.append(x1 x2 y1 y2 z1 z2)
print(sim)

[0, 3, 0, 0, 1]
0
[0, 2, 4, 2, 0]
[2, 5, 1, 2, 0]
1
[0, 2, 4, 2, 0]
[0, 3, 0, 0, 1]
2
[0, 2, 4, 2, 0]
[0, 2, 4, 2, 0]
3
[0, 2, 4, 2, 0]
[0, 2, 0, 3, 2]
4
[0, 2, 4, 2, 0]
[3, 4, 1, 4, 1]
5
[0, 2, 4, 2, 0]
[4, 0, 0, 2, 3]
[]


Manual

In [182]:
import random
def main(data,k):
    centroids = initialize_centroids(data,k)

    while True:
        old_centroids = centroids
        label = get_labels(data,centroids)
        centroids = update_centroids(data,label,k)
        print([old_centroids,label,centroids])
        if should_stop(old_centroids,centroids):
            break
    return label

def initialize_centroids(data,k):
    # x_min = y_min = float('inf')
    # x_max = y_max = float('-inf')
    # for point in data:
    #     x_min = min(point[0],x_min)
    #     x_max = max(point[0],x_max)
    #     y_min = min(point[1],y_min)
    #     y_max = max(point[1],y_max)
    
    centroids = [[4,5,4,3,5],[3,4,1,4,1]]
    # for i in range(k):
    #     centroids.append([random_sample(x_min,x_max),random_sample(y_min,y_max)])
    return centroids

# def random_sample(low,high):
#     return low + (high-low)*random.random()
    
def get_labels(data,centroids):
    labels = []
    # new_dist = get_distance(point_1=[2, 5, 1, 2, 0],point_2=[[5,1,2,3,2],[3,1,2,5,4]])
    # print(new_dist)
    for point in data:
        min_dist = float('inf')
        label = None
        # print('point',point)
        for i, centroid in enumerate(centroids):
            new_dist = get_distance(point,centroid)
            # print('centroid',centroid)
            if min_dist > new_dist:
                min_dist = new_dist
                label = i
        labels.append(label)
    return labels

def get_distance(point_1,point_2):
    # print('point10',point_1[0],'kurang',point_2[0])
    # print('point11',point_1[1],'kurang',point_2[1])
    # print('point12',point_1[2],'kurang',point_2[2])
    # print('point12',point_1[3],'kurang',point_2[3])
    # print('point12',point_1[4],'kurang',point_2[4])
    distance = (((point_1[0] - point_2[0]) ** 2) + ((point_1[1] - point_2[1]) ** 2) + ((point_1[2] - point_2[2]) ** 2) + ((point_1[3] - point_2[3]) ** 2) + ((point_1[4] - point_2[4]) ** 2)) ** 0.5 
    return distance   

def update_centroids(points,labels,k):
    new_centroids = [[0,0,0,0,0] for i in range(k)]
    counts = [0]*k
    for point,label in zip(points,labels):
        new_centroids[label][0] += point[0]
        new_centroids[label][1] += point[1]
        # print(new_centroids[label])
        # print(new_centroids[label][1])
        # print(new_centroids[label][2])
        new_centroids[label][2] += point[2]
        new_centroids[label][3] += point[3]
        new_centroids[label][4] += point[4]
        counts[label] += 1

    for i, (a,b,c,d,e) in enumerate(new_centroids):
        new_centroids[i] = (a/counts[i],b/counts[i],c/counts[i],d/counts[i],e/counts[i])
    return new_centroids

def should_stop(old_centroids,new_centroids,threshold=1e-5):
    total_movement = 0
    for old_point, new_point in zip(old_centroids,new_centroids):
        total_movement += get_distance(old_point,new_point)
    return total_movement < threshold

data = dataarray
k = 2
main(data,k)

[[[4, 5, 4, 3, 5], [3, 4, 1, 4, 1]], [1, 1, 1, 0, 1, 1, 0, 1], [(3.5, 5.0, 3.5, 1.5, 4.0), (1.5, 2.6666666666666665, 1.0, 2.1666666666666665, 1.1666666666666667)]]
[[(3.5, 5.0, 3.5, 1.5, 4.0), (1.5, 2.6666666666666665, 1.0, 2.1666666666666665, 1.1666666666666667)], [1, 1, 1, 0, 1, 1, 0, 1], [(3.5, 5.0, 3.5, 1.5, 4.0), (1.5, 2.6666666666666665, 1.0, 2.1666666666666665, 1.1666666666666667)]]


[1, 1, 1, 0, 1, 1, 0, 1]

In [183]:
Kmeanss = main(data,k)
# print(matrix)
# print(Kmeanss)
cluster0 = []
cluster1 = []
cluster2 = []
for i, j in enumerate(Kmeanss):
    if j == 1:
        cluster1.append(matrix[i])
    elif j == 2:
        cluster2.append(matrix[i])
    else:
        cluster0.append(matrix[i])
print('cluster0=',cluster0)
print('cluster1=',cluster1)
print('cluster2=',cluster2)

[[[4, 5, 4, 3, 5], [3, 4, 1, 4, 1]], [1, 1, 1, 0, 1, 1, 0, 1], [(3.5, 5.0, 3.5, 1.5, 4.0), (1.5, 2.6666666666666665, 1.0, 2.1666666666666665, 1.1666666666666667)]]
[[(3.5, 5.0, 3.5, 1.5, 4.0), (1.5, 2.6666666666666665, 1.0, 2.1666666666666665, 1.1666666666666667)], [1, 1, 1, 0, 1, 1, 0, 1], [(3.5, 5.0, 3.5, 1.5, 4.0), (1.5, 2.6666666666666665, 1.0, 2.1666666666666665, 1.1666666666666667)]]
cluster0= [[4, 5, 4, 3, 5], [3, 5, 3, 0, 3]]
cluster1= [[2, 5, 1, 2, 0], [0, 3, 0, 0, 1], [0, 2, 4, 2, 0], [0, 2, 0, 3, 2], [3, 4, 1, 4, 1], [4, 0, 0, 2, 3]]
cluster2= []


<h2> KNN User Pearson

cek cluster

In [184]:
inputknn = 2
print('user target = ', inputknn)
if inputknn == 1:
    target = matrix[0]
    print(matrix[0])
    if target in cluster0:
        print("user masuk cluster pertama")
        clusterhitung = cluster0
    elif target in cluster1:
        print("user masuk cluster kedua")
        clusterhitung = cluster1
    else:
        print("user masuk cluster ketiga")
        clusterhitung = cluster2
elif inputknn == 2:
    target = matrix[1]
    print(matrix[1])
    if target in cluster0:
        print("user masuk cluster pertama")
        clusterhitung = cluster0
    elif target in cluster1:
        print("user masuk cluster kedua")
        clusterhitung = cluster1
    else:
        print("user masuk cluster ketiga")
        clusterhitung = cluster2
elif inputknn == 3:
    target = matrix[2]
    print(matrix[2])
    if target in cluster0:
        print("user masuk cluster pertama")
        clusterhitung = cluster0
    elif target in cluster1:
        print("user masuk cluster kedua")
        clusterhitung = cluster1
    else:
        print("user masuk cluster ketiga")
        clusterhitung = cluster2
elif inputknn == 4:
    target = matrix[3]
    print(matrix[3])
    if target in cluster0:
        print("user masuk cluster pertama")
        clusterhitung = cluster0
    elif target in cluster1:
        print("user masuk cluster kedua")
        clusterhitung = cluster1
    else:
        print("user masuk cluster ketiga")
        clusterhitung = cluster2
elif inputknn == 6:
    target = matrix[5]
    print(matrix[5])
    if target in cluster0:
        print("user masuk cluster pertama")
        clusterhitung = cluster0
    elif target in cluster1:
        print("user masuk cluster kedua")
        clusterhitung = cluster1
    else:
        print("user masuk cluster ketiga")
        clusterhitung = cluster2
elif inputknn == 7:
    target = matrix[6]
    print(matrix[6])
    if target in cluster0:
        print("user masuk cluster pertama")
        clusterhitung = cluster0
    elif target in cluster1:
        print("user masuk cluster kedua")
        clusterhitung = cluster1
    else:
        print("user masuk cluster ketiga")
        clusterhitung = cluster2
elif inputknn == 8:
    target = matrix[7]
    print(matrix[7])
    if target in cluster0:
        print("user masuk cluster pertama")
        clusterhitung = cluster0
    elif target in cluster1:
        print("user masuk cluster kedua")
        clusterhitung = cluster1
    else:
        print("user masuk cluster ketiga")
        clusterhitung = cluster2
else:
    target = matrix[9]
    print(matrix[9])
    if target in cluster0:
        print("user masuk cluster pertama")
        clusterhitung = cluster0
    elif target in cluster1:
        print("user masuk cluster kedua")
        clusterhitung = cluster1
    else:
        print("user masuk cluster ketiga")
        clusterhitung = cluster2
print(clusterhitung)

user target =  2
[0, 3, 0, 0, 1]
user masuk cluster kedua
[[2, 5, 1, 2, 0], [0, 3, 0, 0, 1], [0, 2, 4, 2, 0], [0, 2, 0, 3, 2], [3, 4, 1, 4, 1], [4, 0, 0, 2, 3]]


In [185]:
# inputknn = 10
# target = []
# for i in range(inputknn):
#   print(i)
#  if target in cluster0:
#     print("user masuk cluster pertama")
#     clusterhitung = cluster0
# else:
#     print("user masuk cluster kedua")
#     clusterhitung = cluster1

# print(target)

In [186]:
average = []
for i in clusterhitung:
    y = sum(i)/len(i)
    average.append(y)
print(average)

[2.0, 0.8, 1.6, 1.4, 2.6, 1.8]


In [187]:
mcr = []
for i in range(len(clusterhitung)):
    data = []
    for j in range(len(clusterhitung[i])):
        data.append(clusterhitung[i][j]-average[i])
    mcr.append(data)
print(mcr)

[[0.0, 3.0, -1.0, 0.0, -2.0], [-0.8, 2.2, -0.8, -0.8, 0.19999999999999996], [-1.6, 0.3999999999999999, 2.4, 0.3999999999999999, -1.6], [-1.4, 0.6000000000000001, -1.4, 1.6, 0.6000000000000001], [0.3999999999999999, 1.4, -1.6, 1.4, -1.6], [2.2, -1.8, -1.8, 0.19999999999999996, 1.2]]


In [188]:
sim = []
